In [2]:
# This class is used for running commands.
import subprocess 

# This is an example of how to run a command. You make a variable to hold the CompletedProcess object.
# completed_dir_command = subprocess.run(["dir"], shell=True, capture_output=True)

# In order to properly print it, you must decode the bytes object of the stdout into a string, or else it will look ugly.
# print(completed_dir_command.stdout.decode("UTF-8"))

# Further documentation is here:
# https://docs.python.org/3/library/subprocess.html#module-subprocess

# Putting completed functions in here
import pokemon_showdown_cli_helper as pcli

In [3]:
def load_and_pack_team(path):
    result = subprocess.run(f"node pokemon-showdown-access.js pack-file {path}", capture_output=True)
    return result.stdout.decode().strip()

# Load wolfe's team
wolfe_packed_team = pcli.get_path_load_and_pack_team("debug11")

# Load John's team
john_packed_team = pcli.get_path_load_and_pack_team("debug12")

In [4]:
# Load wolfe's team
wolfe_json_team = pcli.load_and_json_team("example-teams\\WolfeGlick-worlds-2016.txt")

# Load John's team
john_json_team = pcli.load_and_json_team("example-teams\\JonathanEvans-worlds-2016.txt")

In [ ]:
# All possible move generation
# What if we let pokemon showdown do more of the work for us. Let's try it.

# We need this to use regex
import re

# We need this to get factorials (i'm too lazy to write a dynamic programming solution, plus I don't think our factorials will get that big)
import math

# We need this to generate permutations (I'm too lazy to implement Heap's algorithm on my own)
from itertools import permutations

# We need this for multithreading
import threading

# We need this to collect our results from multi threading
import queue

# For saving and loading
import json
from poke_tree import PokeNodeEncoder

# Way to store the game states
from poke_tree import PokeNode
root = PokeNode("root")

filename = "data.json"

# There really can't be spaces in names. I could validate this data, and I probably will, later.
sim_result = pcli.sim("gen6vgc2016", "Wolfe_Glick", wolfe_packed_team, "Jonathan_Evans", john_packed_team)
#print(sim_result)

# Since the format we entered has teampreview, we know that the 
# last line will always be "teampreview" letting us know what 
# phase we are in.
last_line = sim_result.splitlines()[-1]
print(last_line)
if("teampreview" in last_line):
    # Since we already know we have teampreview, this should be safe.
    pokemon_to_choose = int(re.findall(r"[0-9]", last_line)[0])
    print("We are in team preview phase")
    print(f"We need to choose {pokemon_to_choose} pokemon")

    # Let's just assume that there are 6 pokemon for now
    print(f"We know, using combinatorics, that there are {math.perm(6, pokemon_to_choose)} ways to choose {pokemon_to_choose} from 6")

    # So, let's generate those possibilities.
    # Get's an array from 1 to 6
    possible_choices = list(range(1, 6 + 1))
    # Does 6 permutations 4.
    possible_choices = list(permutations(possible_choices, pokemon_to_choose))
    
    # So, how do we now continue the simulation?
    # We need to write
    # >p1 team 1234(permutation)
    # then 
    # >p2 team 1234(permutation)
    # we can't have spaces.. so. why not use a filler character?
    sim_result = pcli.sim("gen6vgc2016", "Wolfe_Glick", wolfe_packed_team, "Jonathan_Evans", john_packed_team, [">p1 team 1234".replace(" ", "|"),">p2 team 1234".replace(" ", "|")])

    successes = 0

    # Test every permutation
    # This is kinda inherently slow because pokemon-showdown takes like 3 seconds each time.
    # Not to worry, python files are a bit quicker than notebook files.
    threads = []
    result_queue = queue.Queue()
    for possible_choice in possible_choices:
        choice_as_string = pcli.num_tuple_to_string(possible_choice)
        command_to_add = ">p1 team " + choice_as_string

        new_thread = threading.Thread(target=pcli.sim_threading_wrapper, args=("gen6vgc2016", "Wolfe_Glick", wolfe_packed_team, "Jonathan_Evans", john_packed_team, [command_to_add.replace(" ", "|"),">p2 team 1234".replace(" ", "|")], result_queue))
        threads.append(new_thread)

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    results = []
    while not result_queue.empty():
        root.child.append(PokeNode(result_queue.get()))
    for node in root.child:
        if("|turn|1" in node.pokemon_showdown_log):
            successes += 1

    print(successes)

    with open(filename, 'w') as file:
        json.dump(root, file, indent=4, cls=PokeNodeEncoder)


|teampreview|4
We are in team preview phase
We need to choose 4 pokemon
We know, using combinatorics, that there are 360 ways to choose 4 from 6
360
